In [1]:
# 导入包
import os
import surprise
from surprise import Dataset, Reader
from surprise import NormalPredictor, BaselineOnly
from surprise import model_selection, print_perf

In [2]:
help(Dataset.load_builtin)

Help on method load_builtin in module surprise.dataset:

load_builtin(name='ml-100k') method of builtins.type instance
    Load a built-in dataset.
    
    If the dataset has not already been loaded, it will be downloaded and
    saved. You will have to split your dataset using the :meth:`split
    <DatasetAutoFolds.split>` method. See an example in the :ref:`User
    Guide <cross_validate_example>`.
    
    Args:
        name(:obj:`string`): The name of the built-in dataset to load.
            Accepted values are 'ml-100k', 'ml-1m', and 'jester'.
            Default is 'ml-100k'.
    
    Returns:
        A :obj:`Dataset` object.
    
    Raises:
        ValueError: If the ``name`` parameter is incorrect.



## 基本的用法

In [3]:
# 方式二：加载指定数据集
# 导入自己的数据集/实际工作中导入业务数据集
## 直接读取文件
# 指定文件所在的路径(要求给定的文件中只有数据，没有列)
file_path = os.path.expanduser('./datas/ratings.csv')
# 必须给定数据格式(必须给定一个数据读取器，告诉如何读取数据)
reader = Reader(line_format='user item rating timestamp', sep=',')
# 加载数据
data = Dataset.load_from_file(file_path, reader=reader)

In [4]:
# 方式一：加载模式数据（这种方式会从网络上下载数据，速度比较慢）
# 导入surprise默认携带的数据(Movieline数据)
# 默认加载数据，会提示需要下载数据，当数据已经存在的时候就不需要下载啦
# 支持三种参数：'ml-100k', 'ml-1m', and 'jester'
# data = Dataset.load_builtin('ml-100k')

In [5]:
# 可以通过手动的进行数据切分，产生和机器学习交叉验证类似的效果
# data.split(n_folds=3)

## 基于统计的推荐算法

Normal Predictor：核心思想是认为用户对于物品评分是服从高斯分布的一个随机过程

In [6]:
# 构建模型对象
algo = NormalPredictor()

# 模型训练
pref = model_selection.cross_validate(algo, data, measures=['rmse', 'mae', 'fcp'], cv=5, verbose=True)

# 输出最终结果值（评估指标）
print_perf(pref)

Evaluating RMSE, MAE, FCP of algorithm NormalPredictor on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.4345  1.4401  1.4248  1.4341  1.4417  1.4350  0.0059  
MAE (testset)     1.1479  1.1495  1.1374  1.1462  1.1548  1.1471  0.0056  
FCP (testset)     0.4958  0.5000  0.4888  0.4916  0.4890  0.4930  0.0043  
Fit time          0.22    0.24    0.25    0.25    0.24    0.24    0.01    
Test time         0.32    0.28    0.28    0.28    0.22    0.28    0.03    
        Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    
TEST_RMSE1.4345  1.4401  1.4248  1.4341  1.4417  1.4350  
TEST_MAE1.1479  1.1495  1.1374  1.1462  1.1548  1.1471  
TEST_FCP0.4958  0.5000  0.4888  0.4916  0.4890  0.4930  
FIT_TIME0.2219  0.2439  0.2479  0.2459  0.2449  0.2409  
TEST_TIME0.3198  0.2828  0.2838  0.2808  0.2249  0.2784  


In [7]:
# 直接返回随机的评分值
print("用户1对于物品2的评分:%.3f" % algo.predict("1", "2").est)
print("用户1对于物品3的评分:%.3f" % algo.predict("1", "3").est)

用户1对于物品2的评分:4.069
用户1对于物品3的评分:1.605


## 基于统计的基线推荐算法

Baseline Only:算法的思想是认为用户u对于物品i的评分，是相对于平均评分μ的一个偏移；而这个偏移体现在两个方面：<br/>
1. 用户的差异性：用户u相对于其它用户而言，对于物品可能具有比较严格的要求或者比较宽松的要求；当具有比较严格的要求的时候，其实体现的是用户u和其它用户v相比，对于同一个物品i而言，一般r_ui < r_vi;所以可以可以通过找出用户u对所有物品的评分和全局平均评分μ之间的差值来作为用户u在评分一个新物品的时候，会给定一个相对于μ的一个分值。<br/>
2. 物品的差异性：物品i相对于其它物品而言，物品确实可能质量/性能比较低或者比较高的；当物品i的性能/质量比其它的物品差的时候，那么就可以认为物品i的最终评分相对于其它物品应该低一点；

In [8]:
# BaselineOnly的基本用法
bsl_options = {
    'method':'als', # 指定使用何种方式进行计算，默认为als，可选sgd
    'n_epochs':10, # 迭代次数
    'reg_i':25, # b_i计算过程中的正则化项，也就是ppt上的那个λ_2
    'reg_u':10 # b_u计算过程中的正则化项，也就是ppt上的那个λ_3
}
bsl_options = {
    'method':'sgd', 
    'n_epochs':10, # 迭代次数
    'reg':0.02, # 正则化系数
    'learning_rate':0.01 # 学习率，这个和机器学习一样，一般选一个比较小的值
}
algo = BaselineOnly(bsl_options=bsl_options)

# 模型训练
pref = model_selection.cross_validate(algo, data, measures=['rmse', 'mae', 'fcp'], cv=5, verbose=True)

Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Evaluating RMSE, MAE, FCP of algorithm BaselineOnly on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8949  0.8941  0.8913  0.8816  0.8920  0.8908  0.0048  
MAE (testset)     0.6890  0.6890  0.6870  0.6810  0.6886  0.6869  0.0030  
FCP (testset)     0.6567  0.6604  0.6548  0.6536  0.6459  0.6543  0.0048  
Fit time          0.37    0.39    0.36    0.36    0.37    0.37    0.01    
Test time         0.26    0.29    0.26    0.25    0.25    0.26    0.01    


In [9]:
# 获取用户u和物品i之间的推荐系统计算出来的评分
print(algo.predict("1", "1061", 3.0))
# 备注：如果使用algo.estimate进行评分获取的，需要我们将id转换为内部id才可以进行数据读取操作

user: 1          item: 1061       r_ui = 3.00   est = 2.80   {'was_impossible': False}
